In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import re

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search, obtain_SWNUNetwork_input
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
x_data = obtain_SWNUNetwork_input(
    method = "gaussian_random_projection",
    dimension= 30,
    df= df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings= sbert_embeddings,
    k=5,
    features='time_encoding',
    standardise_method=None,
    include_features_in_path=False,
)

x_data["x_data"]["path"].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 30])

## SWNU Network

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = False

In [9]:
split_ids = torch.tensor(df_rumours['timeline_id'].astype(int))

In [10]:
num_epochs = 100
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
bidirectional = True
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

# w=5

In [11]:
size=5

## UMAP

In [12]:
swnu_network_umap_kfold_5, best_swnu_network_umap_kfold_5, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=split_ids,
    k_fold=True,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_5_kfold_best_model.csv


In [13]:
swnu_network_umap_kfold_5.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

/tmp/ipykernel_473982/1978106096.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_5.groupby(["dimensions",


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.687108   
                                                       0.0005         0.691706   
                                                       0.0010         0.692016   
                                          0.2          0.0001         0.686797   
                                                       0.0005         0.690277   
                                                       0.0010         0.693818   
                           (512, 512)     0.1          0.0001         0.686362   
                                                       0.0005         0.689096   
                                                       0.0010         0.689531   
                                          0.2          0.0001         0.688847   
                                                       0.0005         0.692513   
                                                       0.0010         0.690028   
           (12,)           (256, 256)     0.1          0.0001         0.685368   
                                                       0.0005         0.685803   
                                                       0.0010         0.691830   
                                          0.2          0.0001         0.686859   
                                                       0.0005         0.684995   
                                                       0.0010         0.689407   
                           (512, 512)     0.1          0.0001         0.689531   
                                                       0.0005         0.690276   
                                                       0.0010         0.688661   
                                          0.2          0.0001         0.690587   
                                                       0.0005         0.688350   
                                                       0.0010         0.690898   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.662214   
                                                       0.0005         0.664063   
                                                       0.0010         0.664328   
                                          0.2          0.0001         0.661474   
                                                       0.0005         0.663905   
                                                       0.0010         0.664232   
                           (512, 512)     0.1          0.0001         0.660117   
                                                       0.0005         0.660942   
                                                       0.0010         0.661581   
                                          0.2          0.0001         0.663730   
                                                       0.0005         0.661137   
                                                       0.0010         0.661292   
           (12,)           (256, 256)     0.1          0.0001         0.660404   
                                                       0.0005         0.659536   
                                                       0.0010         0.661081   
                                          0.2          0.0001         0.661924   
                                                       0.0005         0.659742   
                                                       0.0010         0.661765   
                           (512, 512)     0.1          0.0001         0.665603   
                                                       0.0005         0.663701   
                                                       0.0010         0.660154   
                                          0.2      

In [14]:
best_swnu_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.688723,0.664666,"[0.754483975301382, 0.574847250509165]",0.662377,"[0.7742908871454436, 0.5504631886884447]",0.668578,"[0.7356651376146789, 0.6014917421417155]",None,0.735409,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.690028,0.668875,"[0.7525665823538165, 0.5851833374906461]",0.666217,"[0.7822455923291061, 0.550187617260788]",0.674995,"[0.7250573394495413, 0.6249334043686734]",None,0.728535,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.693010,0.664017,"[0.7627143062959227, 0.5653206650831355]",0.663363,"[0.7665797856935997, 0.5601464435146444]",0.664739,"[0.7588876146788991, 0.57059136920618]",None,0.737161,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


0.6369639772772482

In [15]:
best_swnu_network_umap_kfold_5["f1"].mean()

0.6658526861723446

In [16]:
best_swnu_network_umap_kfold_5["precision"].mean()

0.6639855857720044

In [17]:
best_swnu_network_umap_kfold_5["recall"].mean()

0.669437767909948

In [18]:
np.stack(best_swnu_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.75658829, 0.57511708])

In [19]:
np.stack(best_swnu_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.77437209, 0.55359908])

In [20]:
np.stack(best_swnu_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.73987003, 0.59900551])

## GRP

In [21]:
swnu_network_grp_kfold_5, best_swnu_network_grp_kfold_5, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=split_ids,
    k_fold=True,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_kfold_5_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_kfold_5_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_kfold_5_focal_2_5_kfold_best_model.csv


In [22]:
swnu_network_grp_kfold_5.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

/tmp/ipykernel_473982/1445305551.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_5.groupby(["dimensions",


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.692513   
                                                       0.0005         0.690898   
                                                       0.0010         0.693259   
                                          0.2          0.0001         0.693632   
                                                       0.0005         0.692140   
                                                       0.0010         0.691892   
                           (512, 512)     0.1          0.0001         0.688040   
                                                       0.0005         0.690587   
                                                       0.0010         0.691395   
                                          0.2          0.0001         0.688599   
                                                       0.0005         0.691208   
                                                       0.0010         0.690276   
           (12,)           (256, 256)     0.1          0.0001         0.697670   
                                                       0.0005         0.691333   
                                                       0.0010         0.692824   
                                          0.2          0.0001         0.697359   
                                                       0.0005         0.695185   
                                                       0.0010         0.693445   
                           (512, 512)     0.1          0.0001         0.692513   
                                                       0.0005         0.692265   
                                                       0.0010         0.690649   
                                          0.2          0.0001         0.695309   
                                                       0.0005         0.692016   
                                                       0.0010         0.693321   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.668100   
                                                       0.0005         0.665210   
                                                       0.0010         0.665359   
                                          0.2          0.0001         0.668154   
                                                       0.0005         0.668837   
                                                       0.0010         0.666446   
                           (512, 512)     0.1          0.0001         0.665159   
                                                       0.0005         0.664251   
                                                       0.0010         0.660635   
                                          0.2          0.0001         0.666780   
                                                       0.0005         0.662728   
                                                       0.0010         0.659779   
           (12,)           (256, 256)     0.1          0.0001         0.672431   
                                                       0.0005         0.669585   
                                                       0.0010         0.668847   
                                          0.2          0.0001         0.672063   
                                                       0.0005         0.671472   
                                                       0.0010         0.670280   
                           (512, 512)     0.1          0.0001         0.669349   
                                                       0.0005         0.666923   
                                                       0.0010         0.664476   
                                          0.2      

In [23]:
best_swnu_network_grp_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.694874,0.667708,"[0.7627192346716917, 0.5726964239102061]",0.666370,"[0.7713280562884784, 0.5614124872057318]",0.669372,"[0.7543004587155964, 0.5844432605221097]",None,0.737296,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.704939,0.679188,"[0.770079883805374, 0.588296488946684]",0.677545,"[0.7803944657050339, 0.5746951219512195]",0.681296,"[0.7600344036697247, 0.6025572722429409]",None,0.734466,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.693197,0.670398,"[0.7570838252656434, 0.5837126960040465]",0.667859,"[0.7801094890510949, 0.5556090515166104]",0.675095,"[0.7353784403669725, 0.6148108684070325]",None,0.730152,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


0.6118406155835983

In [24]:
best_swnu_network_grp_kfold_5["f1"].mean()

0.672431425433941

In [25]:
best_swnu_network_grp_kfold_5["precision"].mean()

0.6705914452863615

In [26]:
best_swnu_network_grp_kfold_5["recall"].mean()

0.6752541173207294

In [27]:
np.stack(best_swnu_network_grp_kfold_5["f1_scores"]).mean(axis=0)

array([0.76329431, 0.58156854])

In [28]:
np.stack(best_swnu_network_grp_kfold_5["precision_scores"]).mean(axis=0)

array([0.77727734, 0.56390555])

In [29]:
np.stack(best_swnu_network_grp_kfold_5["recall_scores"]).mean(axis=0)

array([0.74990443, 0.6006038 ])

# w=11

In [ ]:
size=11

## UMAP

In [ ]:
swnu_network_umap_kfold_11, best_swnu_network_umap_kfold_11, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=split_ids,
    k_fold=True,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_11_kfold_best_model.csv


In [ ]:
swnu_network_umap_kfold_11.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

/tmp/ipykernel_473982/883560493.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_11.groupby(["dimensions",


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.681392   
                                                       0.0005         0.684250   
                                                       0.0010         0.688226   
                                          0.2          0.0001         0.681019   
                                                       0.0005         0.681765   
                                                       0.0010         0.687170   
                           (512, 512)     0.1          0.0001         0.685617   
                                                       0.0005         0.686362   
                                                       0.0010         0.686673   
                                          0.2          0.0001         0.684312   
                                                       0.0005         0.685306   
                                                       0.0010         0.688102   
           (12,)           (256, 256)     0.1          0.0001         0.691768   
                                                       0.0005         0.692078   
                                                       0.0010         0.696924   
                                          0.2          0.0001         0.690339   
                                                       0.0005         0.688413   
                                                       0.0010         0.691706   
                           (512, 512)     0.1          0.0001         0.690152   
                                                       0.0005         0.694377   
                                                       0.0010         0.688910   
                                          0.2          0.0001         0.689469   
                                                       0.0005         0.689779   
                                                       0.0010         0.687294   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.657804   
                                                       0.0005         0.660520   
                                                       0.0010         0.663628   
                                          0.2          0.0001         0.657935   
                                                       0.0005         0.658745   
                                                       0.0010         0.659141   
                           (512, 512)     0.1          0.0001         0.663504   
                                                       0.0005         0.662663   
                                                       0.0010         0.662586   
                                          0.2          0.0001         0.662088   
                                                       0.0005         0.659481   
                                                       0.0010         0.665556   
           (12,)           (256, 256)     0.1          0.0001         0.671118   
                                                       0.0005         0.667554   
                                                       0.0010         0.665697   
                                          0.2          0.0001         0.668509   
                                                       0.0005         0.665085   
                                                       0.0010         0.661841   
                           (512, 512)     0.1          0.0001         0.666494   
                                                       0.0005         0.668073   
                                                       0.0010         0.663885   
                                          0.2      

In [ ]:
best_swnu_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.696738,0.674578,"[0.7594974131559498, 0.5896595208070619]",0.671907,"[0.7839487335978028, 0.5598659003831418]",0.679664,"[0.7365252293577982, 0.6228023441662227]",None,0.743362,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.686300,0.666021,"[0.7483176312247645, 0.5837249567153103]",0.663451,"[0.7821194123163488, 0.5447830101569714]",0.672990,"[0.7173165137614679, 0.6286627597229622]",None,0.738779,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.692265,0.672754,"[0.7526591760299626, 0.5928483353884093]",0.670042,"[0.7882020709130844, 0.55188246097337]",0.680284,"[0.7201834862385321, 0.6403835908364411]",None,0.734331,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [ ]:
best_swnu_network_umap_kfold_11["f1"].mean()

0.6711178388869098

In [ ]:
best_swnu_network_umap_kfold_11["precision"].mean()

0.6684669313901198

In [ ]:
best_swnu_network_umap_kfold_11["recall"].mean()

0.6776456540139041

In [ ]:
np.stack(best_swnu_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.75349141, 0.58874427])

In [ ]:
np.stack(best_swnu_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.78475674, 0.55217712])

In [ ]:
np.stack(best_swnu_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.72467508, 0.63061623])

## GRP

In [ ]:
swnu_network_grp_kfold_11, best_swnu_network_grp_kfold_11, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=split_ids,
    k_fold=True,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_kfold_11_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_kfold_11_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_kfold_11_focal_2_11_kfold_best_model.csv


In [ ]:
swnu_network_grp_kfold_11.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

/tmp/ipykernel_473982/2872978494.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_11.groupby(["dimensions",


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.692016   
                                                       0.0005         0.687978   
                                                       0.0010         0.687916   
                                          0.2          0.0001         0.694191   
                                                       0.0005         0.686735   
                                                       0.0010         0.690774   
                           (512, 512)     0.1          0.0001         0.687232   
                                                       0.0005         0.686921   
                                                       0.0010         0.690339   
                                          0.2          0.0001         0.688288   
                                                       0.0005         0.687791   
                                                       0.0010         0.692203   
           (12,)           (256, 256)     0.1          0.0001         0.696676   
                                                       0.0005         0.690774   
                                                       0.0010         0.691768   
                                          0.2          0.0001         0.696614   
                                                       0.0005         0.688164   
                                                       0.0010         0.689779   
                           (512, 512)     0.1          0.0001         0.692824   
                                                       0.0005         0.695309   
                                                       0.0010         0.686052   
                                          0.2          0.0001         0.692451   
                                                       0.0005         0.693445   
                                                       0.0010         0.687978   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.669438   
                                                       0.0005         0.664051   
                                                       0.0010         0.662034   
                                          0.2          0.0001         0.671391   
                                                       0.0005         0.663739   
                                                       0.0010         0.663969   
                           (512, 512)     0.1          0.0001         0.664655   
                                                       0.0005         0.660631   
                                                       0.0010         0.664403   
                                          0.2          0.0001         0.665959   
                                                       0.0005         0.663585   
                                                       0.0010         0.665570   
           (12,)           (256, 256)     0.1          0.0001         0.674780   
                                                       0.0005         0.669440   
                                                       0.0010         0.667884   
                                          0.2          0.0001         0.674524   
                                                       0.0005         0.667169   
                                                       0.0010         0.666833   
                           (512, 512)     0.1          0.0001         0.670499   
                                                       0.0005         0.669359   
                                                       0.0010         0.663558   
                                          0.2      

In [ ]:
best_swnu_network_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.691892,0.668822,"[0.7562306444477216, 0.5814130159534059]",0.666338,"[0.7786213179471606, 0.5540540540540541]",0.673353,"[0.7350917431192661, 0.6116142781033564]",None,0.755088,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.696925,0.672792,"[0.7616534740545294, 0.583930399181167]",0.670518,"[0.7792441511697661, 0.5617922205809945]",0.676362,"[0.7448394495412844, 0.6078849227490677]",None,0.736487,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.686486,0.662037,"[0.7529377203290247, 0.5711371749107598]",0.659833,"[0.7719879518072289, 0.5476772616136919]",0.665751,"[0.7348050458715596, 0.5966968566862014]",None,0.746462,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [ ]:
best_swnu_network_grp_kfold_11["f1"].mean()

0.6678837381461014

In [ ]:
best_swnu_network_grp_kfold_11["precision"].mean()

0.6655628261954827

In [ ]:
best_swnu_network_grp_kfold_11["recall"].mean()

0.6718220493451227

In [ ]:
np.stack(best_swnu_network_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.75694061, 0.57882686])

In [ ]:
np.stack(best_swnu_network_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.77661781, 0.55450785])

In [ ]:
np.stack(best_swnu_network_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.73824541, 0.60539869])

# w=20

In [11]:
size=20

## UMAP

In [ ]:
swnu_network_umap_kfold_20, best_swnu_network_umap_kfold_20, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=split_ids,
    k_fold=True,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_20_kfold_best_model.csv


In [ ]:
swnu_network_umap_kfold_20.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

/tmp/ipykernel_473982/209270084.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_20.groupby(["dimensions",


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.677043   
                                                       0.0005         0.680149   
                                                       0.0010         0.690711   
                                          0.2          0.0001         0.679031   
                                                       0.0005         0.672942   
                                                       0.0010         0.677043   
                           (512, 512)     0.1          0.0001         0.680460   
                                                       0.0005         0.678658   
                                                       0.0010         0.674185   
                                          0.2          0.0001         0.682199   
                                                       0.0005         0.671575   
                                                       0.0010         0.683628   
           (12,)           (256, 256)     0.1          0.0001         0.689469   
                                                       0.0005         0.690152   
                                                       0.0010         0.684063   
                                          0.2          0.0001         0.688226   
                                                       0.0005         0.690276   
                                                       0.0010         0.685430   
                           (512, 512)     0.1          0.0001         0.690339   
                                                       0.0005         0.685617   
                                                       0.0010         0.689096   
                                          0.2          0.0001         0.690152   
                                                       0.0005         0.685865   
                                                       0.0010         0.685244   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.653324   
                                                       0.0005         0.654099   
                                                       0.0010         0.665884   
                                          0.2          0.0001         0.656713   
                                                       0.0005         0.647402   
                                                       0.0010         0.650279   
                           (512, 512)     0.1          0.0001         0.656596   
                                                       0.0005         0.654587   
                                                       0.0010         0.649347   
                                          0.2          0.0001         0.658435   
                                                       0.0005         0.647917   
                                                       0.0010         0.656960   
           (12,)           (256, 256)     0.1          0.0001         0.669961   
                                                       0.0005         0.668843   
                                                       0.0010         0.661930   
                                          0.2          0.0001         0.669451   
                                                       0.0005         0.668330   
                                                       0.0010         0.661311   
                           (512, 512)     0.1          0.0001         0.669009   
                                                       0.0005         0.659846   
                                                       0.0010         0.664988   
                                          0.2      

In [ ]:
best_swnu_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.688910,0.662834,"[0.7565990958144961, 0.5690679060160083]",0.661033,"[0.7699614128821609, 0.5521042084168337]",0.665400,"[0.7436926605504587, 0.5871070857751731]",None,0.762367,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.686114,0.665856,"[0.7481304217768473, 0.5835806132542036]",0.663291,"[0.782051282051282, 0.5445316105214583]",0.672846,"[0.7170298165137615, 0.6286627597229622]",None,0.821135,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.695806,0.676300,"[0.7557617479796468, 0.5968379446640316]",0.673485,"[0.7905447714464621, 0.5564256103178259]",0.683745,"[0.7239105504587156, 0.6435801811401172]",None,0.769376,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [ ]:
best_swnu_network_umap_kfold_20["f1"].mean()

0.6683296215842057

In [ ]:
best_swnu_network_umap_kfold_20["precision"].mean()

0.6659364826060038

In [ ]:
best_swnu_network_umap_kfold_20["recall"].mean()

0.6739971756935313

In [ ]:
np.stack(best_swnu_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.75349709, 0.58316215])

In [ ]:
np.stack(best_swnu_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.78085249, 0.55102048])

In [ ]:
np.stack(best_swnu_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.72821101, 0.61978334])

## GRP

In [ ]:
swnu_network_grp_kfold_20, best_swnu_network_grp_kfold_20, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=split_ids,
    k_fold=True,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_kfold_20_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swnu_network_grp_kfold_20.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

In [ ]:
best_swnu_network_grp_kfold_20

In [ ]:
best_swnu_network_grp_kfold_20["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_20["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_20["recall_scores"]).mean(axis=0)

# w=35

In [ ]:
size=35

## UMAP

In [ ]:
swnu_network_umap_kfold_35, best_swnu_network_umap_kfold_35, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=split_ids,
    k_fold=True,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

In [ ]:
swnu_network_umap_kfold_35.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

In [ ]:
best_swnu_network_umap_kfold_35

In [ ]:
best_swnu_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swnu_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swnu_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
swnu_network_grp_kfold_35, best_swnu_network_grp_kfold_35, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=split_ids,
    k_fold=True,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_kfold_35_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

In [ ]:
swnu_network_grp_kfold_35.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

In [ ]:
best_swnu_network_grp_kfold_35

In [ ]:
best_swnu_network_grp_kfold_35["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_35["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_35["recall_scores"]).mean(axis=0)